# Interactions - Lab

## Introduction

In this lab, you'll explore interactions in the Ames Housing dataset.

## Objectives

You will be able to:
- Implement interaction terms in Python using the `sklearn` and `statsmodels` packages 
- Interpret interaction variables in the context of a real-world problem 

## Build a baseline model 

You'll use a couple of built-in functions, which we imported for you below: 

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

If you still want to build a model in the end, you can do that, but this lab will just focus on finding meaningful insights in interactions and how they can improve $R^2$ values.

In [13]:
regression = LinearRegression()

Create a baseline model which includes all the variables we selected from the Ames housing data set to predict the house prices. Then use 10-fold cross-validation and report the mean $R^2$ value as the baseline $R^2$.

In [14]:
ames = pd.read_csv('ames.csv')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

ames_cont = ames[continuous]

# log features
log_names = [f'{column}_log' for column in ames_cont.columns]

ames_log = np.log(ames_cont)
ames_log.columns = log_names

# normalize (subract mean and divide by std)

def normalize(feature):
    return (feature - feature.mean()) / feature.std()

ames_log_norm = ames_log.apply(normalize)

# one hot encode categoricals
ames_ohe = pd.get_dummies(ames[categoricals], prefix=categoricals)

preprocessed = pd.concat([ames_cont, ames_ohe], axis=1)

X = preprocessed.drop('SalePrice', axis=1)
y = preprocessed['SalePrice']

crossvalidation = KFold(n_splits=10, shuffle=True, random_state=1)
baseline = np.mean(cross_val_score(regression, X, y, scoring='r2', cv=crossvalidation))

baseline

0.7773817128043181

## See how interactions improve your baseline

Next, create all possible combinations of interactions, loop over them and add them to the baseline model one by one to see how they affect the $R^2$. We'll look at the 3 interactions which have the biggest effect on our $R^2$, so print out the top 3 combinations.

You will create a `for` loop to loop through all the combinations of 2 predictors. You can use `combinations` from itertools to create a list of all the pairwise combinations. To find more info on how this is done, have a look [here](https://docs.python.org/2/library/itertools.html).

Since there are so many different neighbourhoods we will exclude

In [15]:
preprocessed.columns

Index(['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice', 'BldgType_1Fam',
       'BldgType_2fmCon', 'BldgType_Duplex', 'BldgType_Twnhs',
       'BldgType_TwnhsE', 'KitchenQual_Ex', 'KitchenQual_Fa', 'KitchenQual_Gd',
       'KitchenQual_TA', 'SaleType_COD', 'SaleType_CWD', 'SaleType_Con',
       'SaleType_ConLD', 'SaleType_ConLI', 'SaleType_ConLw', 'SaleType_New',
       'SaleType_Oth', 'SaleType_WD', 'MSZoning_C (all)', 'MSZoning_FV',
       'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM', 'Street_Grvl',
       'Street_Pave', 'Neighborhood_Blmngtn', 'Neighborhood_Blueste',
       'Neighborhood_BrDale', 'Neighborhood_BrkSide', 'Neighborhood_ClearCr',
       'Neighborhood_CollgCr', 'Neighborhood_Crawfor', 'Neighborhood_Edwards',
       'Neighborhood_Gilbert', 'Neighborhood_IDOTRR', 'Neighborhood_MeadowV',
       'Neighborhood_Mitchel', 'Neighborhood_NAmes', 'Neighborhood_NPkVill',
       'Neighborhood_NWAmes', 'Neighborhood_NoRidge', 'Neighborhood_NridgHt',
       'Neighborhood_OldTown', 'Ne

In [5]:
from itertools import combinations

In [6]:
preprocessed_new = [col for col in preprocessed.columns if 'Neighborhood' not in col]

In [7]:
preprocessed_new

['LotArea',
 '1stFlrSF',
 'GrLivArea',
 'SalePrice',
 'BldgType_1Fam',
 'BldgType_2fmCon',
 'BldgType_Duplex',
 'BldgType_Twnhs',
 'BldgType_TwnhsE',
 'KitchenQual_Ex',
 'KitchenQual_Fa',
 'KitchenQual_Gd',
 'KitchenQual_TA',
 'SaleType_COD',
 'SaleType_CWD',
 'SaleType_Con',
 'SaleType_ConLD',
 'SaleType_ConLI',
 'SaleType_ConLw',
 'SaleType_New',
 'SaleType_Oth',
 'SaleType_WD',
 'MSZoning_C (all)',
 'MSZoning_FV',
 'MSZoning_RH',
 'MSZoning_RL',
 'MSZoning_RM',
 'Street_Grvl',
 'Street_Pave']

In [16]:
# List of Iteractions

list_r2 = []

for i,j in combinations(X.columns,2):
    preprocessed['new'] = preprocessed[i] * preprocessed[j]
    X = preprocessed.drop('SalePrice', axis=1)
    y = preprocessed['SalePrice']
    crossvalidation = KFold(n_splits=10, shuffle=True, random_state=1)
    score = np.mean(cross_val_score(regression, X, y, scoring="r2", cv=crossvalidation))
    if score>baseline:
        list_r2.append([score, i, j])
        

In [17]:
list_r2

[[0.7785627748247997, 'LotArea', 'BldgType_1Fam'],
 [0.7786003663406829, 'LotArea', 'BldgType_2fmCon'],
 [0.777824460199203, 'LotArea', 'SaleType_Con'],
 [0.777487685502375, 'LotArea', 'SaleType_ConLI'],
 [0.7931246124301738, 'LotArea', 'SaleType_New'],
 [0.7780175799515525, 'LotArea', 'SaleType_Oth'],
 [0.7906354461190326, 'LotArea', 'SaleType_WD'],
 [0.7780360916913185, 'LotArea', 'Street_Grvl'],
 [0.7780076249845522, 'LotArea', 'Street_Pave'],
 [0.7773937855773132, 'LotArea', 'Neighborhood_Blueste'],
 [0.7784720543767641, 'LotArea', 'Neighborhood_BrkSide'],
 [0.8093314939294878, 'LotArea', 'Neighborhood_Edwards'],
 [0.7783501453206385, 'LotArea', 'Neighborhood_Mitchel'],
 [0.7783820923640901, 'LotArea', 'Neighborhood_NoRidge'],
 [0.7837853111901494, 'LotArea', 'Neighborhood_NridgHt'],
 [0.778059635795288, 'LotArea', 'Neighborhood_OldTown'],
 [0.7774192053512542, 'LotArea', 'Neighborhood_Sawyer'],
 [0.7780438658290307, 'LotArea', 'Neighborhood_SawyerW'],
 [0.7777232679879511, '1stFlr

In [26]:
for i in list_r2:
    if i[0] > 0.8:
        print(i)

[0.8093314939294878, 'LotArea', 'Neighborhood_Edwards']
[0.8033414554230696, '1stFlrSF', 'Neighborhood_Edwards']
[0.8077114235220242, 'GrLivArea', 'Neighborhood_Edwards']


It looks like the top interactions involve the Neighborhood_Edwards feature so lets add the interaction between LotArea and Edwards to our model.

We can interpret this feature as the relationship between LotArea and SalePrice when the house is in Edwards or not.

## Visualize the Interaction

Separate all houses that are located in Edwards and those that are not. Run a linear regression on each population against `SalePrice`. Visualize the regression line and data points with price on the y axis and LotArea on the x axis.

In [ ]:
# Visualization code here

## Build a final model with interactions

Use 10-fold cross-validation to build a model using the above interaction. 

In [ ]:
# code here

Our $R^2$ has increased considerably! Let's have a look in `statsmodels` to see if this interactions are significant.

In [ ]:
# code here

What is your conclusion here?

In [ ]:
# formulate your conclusion

## Summary

You should now understand how to include interaction effects in your model! As you can see, interactions can have a strong impact on linear regression models, and they should always be considered when you are constructing your models.